# Programming Machine Learning Lab
# Exercise 9

**General Instructions:**

1. You need to submit the PDF as well as the filled notebook file.
1. Name your submissions by prefixing your matriculation number to the filename. Example, if your MR is 12345 then rename the files as **"12345_Exercise_9.xxx"**
1. Complete all your tasks and then do a clean run before generating the final PDF. (_Clear All Ouputs_ and _Run All_ commands in Jupyter notebook)

**Exercise Specific instructions::**

1. You are allowed to use only NumPy and Pandas (unless stated otherwise). You can use any library for visualizations.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import os
import imageio
from sklearn.utils import shuffle
from sklearn.datasets import load_digits
import warnings
warnings.filterwarnings("ignore")

### Part 1

In this part, we will code a perceptron. It is simply a single node neural network which processes weighted inputs and performs binary classification. 


- Read up on perceptron algorithm. (https://en.wikipedia.org/wiki/Perceptron#Learning_algorithm_for_a_single-layer_perceptron).
- Create an object class caled perceptron. 
- Train your perceptron using the following different datasets and report the test losses.
- Create an animation of how the decision boundary is updated over the iterations. *You can use any library for this viualization*
- We will use toy datasets for the problem. Set aside 20% of samples from each dataset for testing.
    - **Xlin_sep.npy** and **ylin_sep.npy**. This dataset is linearly separable. Run your algorithm for this data, and you should achieve 100% train and test accuracies!
    - **Xlinnoise_sep.npy** and **ylinnoise_sep.npy**. This dataset is not linearly separable and contains noise. Run your algorithm for this data and observe what happens to the decision boundary in the animation. You should get a test accuracy over 80%.
    - **circles_x.npy** and **circles_y.npy**. This dataset is non-linear. Devise a strategy to make the dataset separable linearly. *(Hint: Polynomial Features)*. Plot the decision boundary showing how the two classes are separated.



In [3]:
## funtion for normalizing the features
def normalize_features(X,append=True):
    X = (X - np.mean(X, 0)) / np.std(X, 0) #normalize the features
    if append:
        X = np.append(np.ones(X.shape[0]).reshape(-1,1),X,1) #append column of ones for intercept
    return X

In [4]:
class Perceptron:
    def __init__ (self, X, y, iterations=5):
        # Initialize Perceptron with input features X, target labels y, and the number of iterations
        self.X = X
        self.y = y.reshape(-1,1)  # Reshape target labels for consistency
        self.m, self.n = self.X.shape  # Number of examples (m) and features (n)
        self.i = iterations  # Number of max iterations
    
    def fit(self, lr=0.1, tolerance=1e-3):
        # Initialize random weight vector
        new_W = np.ones(shape=(self.n+1,1))
        # To check if the weight update becomes insignificant
        old_W = new_W.copy()
        # To keep track of how the weight values change over iterations
        w_hist = []
        
        # Iterating through max iterations
        for epoch in range(self.i):
            accuracy = 0
            iter_count = 0
            for ind, x_i in enumerate(self.X):
                # Add bias term to input
                x_i = np.insert(x_i, 0, 1).reshape(-1,1)
                # Apply step activation function to the dot product of input and weights
                y_hat = self.activation(np.dot(x_i.T, new_W)) 
                # Calculate accuracy over the iteration rounds
                accuracy += self.accuracy(self.y[ind], y_hat) 
                # Append iteration count
                iter_count += 1
                # Update weights using the perceptron learning rule
                if (np.squeeze(y_hat) - self.y[ind]) != 0:
                    new_W += lr * ((self.y[ind] - y_hat) / 2 * x_i)

            # Store the history of the weight vector after each epoch
            w_hist.append(old_W)
            
            # Print the results after every 10 epochs
            if epoch % 10 == 0 or epoch == self.i - 1:
                print('Epoch {}/{}'.format(epoch, self.i - 1),':', 'Train Accuracy: {:.4f}'.format(accuracy / iter_count)) 

            # The new weights become old for the next iteration
            old_W = new_W
            
        # Store final weights and history of weights for visualization purposes
        self.W = old_W
        self.hist = w_hist
    
    def activation(self, z):
        # Step activation function: 1 for z > 0, -1 otherwise
        return np.where(z > 0, 1, -1)     
    
    def predict(self, X):
        # Add intercept term to input
        X = np.hstack((np.ones((X.shape[0],1)), X))
        # Calculate predictions using the learned weights
        self.pred = self.activation(np.dot(X, self.W).reshape(1, -1)) 
        return self.pred
    
    def accuracy(self, y_true, y_pred):
        # Calculate accuracy by counting the number of correct predictions
        accuracy = np.sum(y_true == y_pred)
        return accuracy

In [5]:
#Linearly Separable Data

#for loading the data
with open('Xlin_sep.npy','rb') as f:
    X = np.load(f)
with open('ylin_sep.npy','rb') as f:
    y = np.load(f)
    

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#normalize training set
X_train_normalized = normalize_features(X_train, append=False)

#normalizing test set
X_test_normalized = normalize_features(X_test,append=False)

#making the model
p = Perceptron(X_train_normalized, y_train)
#fitting the perceptron
p.fit()

#to get predictions on train and test set
preds_train = p.predict(X_train_normalized)
preds_test = p.predict(X_test_normalized)

print('The accuracy on Train Set is',p.accuracy(y_train,preds_train)/len(y_train))
print('The accuracy on Test Set is',p.accuracy(y_test,preds_test)/len(y_test))

Epoch 0/4 : Train Accuracy: 0.9067
Epoch 4/4 : Train Accuracy: 1.0000
The accuracy on Train Set is 1.0
The accuracy on Test Set is 1.0


In [5]:
class Visualize:
    
    def __init__(self, X, y, W):
        self.X = X
        self.y = y
        self.W = W
    
    def create_frame(self, w, iteration, name='Linearly Separable'):
    
        if not os.path.exists(name):
            os.mkdir(name)
        markers = ('s', 'o')
        colors = ('red', 'blue')
        cmap = plt.cm.RdBu
        fig, ax = plt.subplots(figsize=(8, 6))
        # Plot the decision surface
        x1_min, x1_max = self.X[:, 0].min() - 1, self.X[:, 0].max() + 1
        x2_min, x2_max = self.X[:, 1].min() - 1, self.X[:, 1].max() + 1
        xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.01),
                               np.arange(x2_min, x2_max, 0.01))
        
        X = np.array([xx1.ravel(), xx2.ravel()]).T
        X = np.hstack((np.ones((X.shape[0],1)), X))
            
        Z = np.dot(X, w).reshape(1, -1)
        Z = np.where(Z > 0, 1, -1)
        Z = Z.reshape(xx1.shape)
        
        ax.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)

        # Plot all samples
        for idx, cl in enumerate(np.unique(y)):
            ax.scatter(x=self.X[self.y == cl, 0], y=self.X[self.y == cl, 1],
                       alpha=0.8, c=colors[idx],
                       marker=markers[idx], label=f'Class {cl}')


        ax.set_xlabel('Feature 1')
        ax.set_ylabel('Feature 2')
        ax.set_title(f'Decision Boundary of {name} at iteration {iteration}')

        ax.legend(loc='upper left')
        plt.savefig(f'./{name}/img_{iteration}.png', 
                        transparent = False,  
                        facecolor = 'white'
                       )
        plt.close()

    def create_animation(self,name):

        for t,w in enumerate(self.W):
            self.create_frame(w,t,name)

        frames = []
        for t in range(len(self.W)):
            image = imageio.v2.imread(f'./{name}/img_{t}.png')
            frames.append(image)

        imageio.mimsave(f'./{name}.gif', frames, fps = 5)

In [7]:
#history of weight vectors
W = p.hist

In [8]:
Visualize(X_train_normalized,y_train,W).create_animation('train_lin_sep_')
Visualize(X_test_normalized,y_test,W).create_animation('test_lin_sep_')

<img src="train_lin_sep_.gif" width="350" align="center">

<img src="test_lin_sep_.gif" width="350" align="center">

### 2. Data with Noise

In [9]:
#for loading the data
with open('Xlinnoise_sep.npy','rb') as f:
    X = np.load(f)
with open('ylinnoise_sep.npy','rb') as f:
    y = np.load(f)

In [10]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=14)

#normalize training set
X_train_normalized = normalize_features(X_train,append=False)

#normalizing test set
X_test_normalized = normalize_features(X_test,append=False)

In [11]:
#making the model
p = Perceptron(X_train_normalized,y_train,iterations=20)
#fitting the perceptron
p.fit()

Epoch 0/19 : Train Accuracy: 0.7933
Epoch 10/19 : Train Accuracy: 0.7667
Epoch 19/19 : Train Accuracy: 0.7667


In [12]:
#to get predictions on train and test set
preds_train = p.predict(X_train_normalized)
preds_test = p.predict(X_test_normalized)

In [13]:
print('The accuracy on Train Set is',p.accuracy(y_train,preds_train)/len(y_train))
print('The accuracy on Test Set is',p.accuracy(y_test,preds_test)/len(y_test))

The accuracy on Train Set is 0.82
The accuracy on Test Set is 0.78


In [14]:
#history of weight vectors
W = p.hist

In [15]:
Visualize(X_train_normalized,y_train,W).create_animation('train_linnoise_sep_')
Visualize(X_test_normalized,y_test,W).create_animation('test_linnoise_sep_')

<img src="train_linnoise_sep_.gif" width="350" align="center">

<img src="test_linnoise_sep_.gif" width="350" align="center">

### 3. Circle Data (Non-Linear)

In [16]:
#for loading the data
with open('circles_x.npy','rb') as f:
    X = np.load(f)
with open('circles_y.npy','rb') as f:
    y = np.load(f)

Since the data is not linearly separable, we will add polynomial features into the data. This means instead of giving the perceptron x1 and x2 as input we will now give it  $ x_1^2, x_2^2$

In [17]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(np.square(X), y, test_size=0.25, random_state=42)

#normalize training set
X_train_normalized = normalize_features(X_train,append=False)

#normalizing test set
X_test_normalized = normalize_features(X_test,append=False)

In [18]:
#making the model
p = Perceptron(X_train_normalized,y_train,iterations=20)
#fitting the perceptron
p.fit()

Epoch 0/19 : Train Accuracy: 0.7933
Epoch 10/19 : Train Accuracy: 1.0000
Epoch 19/19 : Train Accuracy: 1.0000


In [19]:
#to get predictions on train and test set
preds_train = p.predict(X_train_normalized)
preds_test = p.predict(X_test_normalized)

In [20]:
print('The accuracy on Train Set is',p.accuracy(y_train,preds_train)/len(y_train))
print('The accuracy on Test Set is',p.accuracy(y_test,preds_test)/len(y_test))

The accuracy on Train Set is 1.0
The accuracy on Test Set is 1.0


In [21]:
#history of weight vectors
W = p.hist

In [22]:
Visualize(X_train_normalized,y_train,W).create_animation('train_circle_')
Visualize(X_test_normalized,y_test,W).create_animation('test_circle_')

<img src="train_circle_.gif" width="350" align="center">

<img src="test_circle_.gif" width="350" align="center">

### Part 2

In this part, we will create a feed-forward neural network

- Load the MNIST classification dataset using sklearn. Split the data into train and test datasets (80-20 split).
- Implement a neural network with forward propagation and backpropagation **from scratch**.
- Use Stochastic Gradient Descent as the optimizer and Cross-entropy as Loss.
- You model class should be flexible in terms of
    - Number of layers
    - Number of hidder parameters.
    - Activation function for each layer (SoftMax, ReLU or tanh)
- Now create a training function that takes the neural network and training data as inputs and updates the weights of the network. This function should also take in the learning rate, number of epochs, and batchsize as input.
- Try out different hyperparameters to train your model and try to achieve >90% test accuracy. 

*Hints:*
- Flatten the MNIST data from 2D to 1D.
- Use *He weights initialization* for weights. *The He initialization calculates the starting weights as randomly generated matrices using a Gaussian probability distribution with a mean of 0.0 and a standard deviation of sqrt(2/n), where n is the number of inputs to the layer.*

In [23]:
### Write your code here

# Activation functions
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def softmax(x):
    exps = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exps / np.sum(exps, axis=-1, keepdims=True)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x)**2

# Cross-entropy gradient, the gradient of the loss with respect to the output
def cross_entropy_gradient(y_pred, y_true):
    n_samples = y_true.shape[0]
    res = y_pred - y_true
    return res / n_samples

class NeuralNetwork:
    def __init__(self, n_inputs, n_outputs, n_hidden, hidden_dims, activation_funcs):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.n_hidden = n_hidden  # hidden layers
        self.hidden_dims = hidden_dims  # neurons in each hidden layer

        # list for the number of neurons in each layer
        layers = [self.n_inputs] + self.hidden_dims + [self.n_outputs]
        # weights of each layer
        self.weights = []
        # bias for each layer
        self.biases = []
        # activation func for each layer
        self.activations = []
        # derivatives of the weights
        self.dW = []
        # derivatives of the biases
        self.db = []

        for i in range(self.n_hidden + 1):
            # weights for each layer with "He" initialization
            self.weights.append(np.random.randn(layers[i], layers[i+1]) * np.sqrt(2. / layers[i]))

            # biases for each layer with zeros
            self.biases.append(np.zeros((1, layers[i+1])))

            # activation function for each layer
            self.activations.append(activation_funcs[i])

            # derivatives of the weights and biases with zeros
            self.dW.append(np.zeros((layers[i], layers[i+1])))
            self.db.append(np.zeros((1, layers[i+1])))

    def forward(self, X):
        # weighted sums for each layer
        self.A = []
        # activations for each layer
        self.Z = []

        # For each layer
        for i in range(self.n_hidden + 1):
            # weighted sum of the inputs
            if i == 0:
                self.A.append(np.dot(X, self.weights[i]) + self.biases[i])
            else:
                self.A.append(np.dot(self.Z[i-1], self.weights[i]) + self.biases[i])

            # Apply the activation function
            if self.activations[i] == 'relu':
                self.Z.append(relu(self.A[i]))
            elif self.activations[i] == 'softmax':
                self.Z.append(softmax(self.A[i]))
            elif self.activations[i] == 'tanh':
                self.Z.append(tanh(self.A[i]))

        # output of the last layer
        return self.Z[-1]

    def backward(self, X, y, output):
        # gradient of the loss with respect to the output
        # Note: it's also the gradient for softmax activation layer
        # which is why it's not in the loop below
        self.dZ = cross_entropy_gradient(output, y)

        # gradient of the loss with respect to the weight and bias of the last layer
        self.dW[-1] = np.dot(self.Z[-2].T, self.dZ)
        self.db[-1] = np.sum(self.dZ, axis=0, keepdims=True)

        # For each layer in reverse order
        for i in range(self.n_hidden-1, -1, -1):
            # gradient of the loss with respect to the weighted sum
            if self.activations[i] == 'relu':
                self.dZ = np.dot(self.dZ, self.weights[i+1].T) * relu_derivative(self.A[i])
            elif self.activations[i] == 'tanh':
                self.dZ = np.dot(self.dZ, self.weights[i+1].T) * tanh_derivative(self.A[i])

            # gradient of the loss with respect to the weights and biases
            if i > 0:
                self.dW[i-1] = np.dot(self.Z[i-1].T, self.dZ)
            else:
                self.dW[i] = np.dot(X.T, self.dZ)
            self.db[i] = np.sum(self.dZ, axis=0, keepdims=True)

    def train(self, X, y, n_epochs, learning_rate, batch_size):
        # num of batches based on batch size
        n_batches = X.shape[0] // batch_size

        for epoch in range(n_epochs):
            # for each batch
            for batch in range(n_batches):
                # batch from data
                X_batch = X[batch * batch_size:(batch + 1) * batch_size]
                y_batch = y[batch * batch_size:(batch + 1) * batch_size]

                # Forward propagation to get output layer
                output = self.forward(X_batch)

                # Backward propagation to get gradients of the loss function
                self.backward(X_batch, y_batch, output)

                # Update weights and biases using gradients and learning rate
                for i in range(self.n_hidden + 1):
                    self.weights[i] -= learning_rate * self.dW[i]
                    self.biases[i] -= learning_rate * self.db[i]

            # Print loss and accuracy every 10 epochs
            if epoch % 10 == 0:
                predictions = self.predict(X)
                loss = self.calculate_loss(predictions, y)
                accuracy = self.calculate_accuracy(predictions, y)
                print(f"Epoch {epoch}: Loss = {loss}, Accuracy = {accuracy}")

    def calculate_loss(self, predictions, y):
        # If y is not one-hot encoded, convert it to one-hot encoding
        if len(y.shape) == 1:
            y_one_hot = np.zeros_like(predictions)
            y_one_hot[np.arange(len(y)), y] = 1
        else:
            y_one_hot = y

        # Cross-entropy loss
        epsilon = 1e-15
        predictions = np.clip(predictions, epsilon, 1 - epsilon)
        loss = -np.sum(y_one_hot * np.log(predictions)) / len(y)
        return loss


    def calculate_accuracy(self, predictions, y):
        # If y is not one-hot encoded, convert it to one-hot encoding
        if len(y.shape) == 1:
            y_one_hot = np.zeros_like(predictions)
            y_one_hot[np.arange(len(y)), y] = 1
        else:
            y_one_hot = y

        # Accuracy calculation
        correct_predictions = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_one_hot, axis=1))
        accuracy = correct_predictions / len(y)
        return accuracy
    
    def predict(self, X):
        output = self.forward(X)
        return output

In [24]:
# MNIST dataset mini version, data already flatterned
digits = load_digits()
#to separate in the features and labels
X = digits['data']
y = digits['target']
#scalar for scaling the features aka pixels
scalar = StandardScaler()
X = scalar.fit_transform(X) 
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#train valid split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# One-hot encode the labels, 10 posible lables (0 to 9)
one_hot = OneHotEncoder(sparse_output=False)
y = one_hot.fit_transform(np.array(y).reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
# 2 layers to capture complex data relu is most popular layer in all nn's and softmax for classification
nn = NeuralNetwork(n_inputs=64, n_outputs=10, n_hidden=2, hidden_dims=[32, 32], activation_funcs=['relu', 'relu', 'softmax'])
nn.train(X_train, y_train, n_epochs=50, learning_rate=0.01, batch_size=32)

# test accuracy
y_pred = np.argmax(nn.predict(X_test), axis=1)

# convert encoded y_test to class labels
y_test_labels = np.argmax(y_test, axis=1)
# mean of the comparison between predicted and true labels
test_accuracy = np.mean(y_test_labels  == y_pred)
print(f'Test accuracy: {test_accuracy:.2%}')

Epoch 0: Loss = 2.166656993990253, Accuracy = 0.2199025748086291
Epoch 10: Loss = 0.6389826570078291, Accuracy = 0.8531663187195546
Epoch 20: Loss = 0.3265398462773961, Accuracy = 0.9248434237995825
Epoch 30: Loss = 0.2108951207217554, Accuracy = 0.9491997216423104
Epoch 40: Loss = 0.15303274940553013, Accuracy = 0.9659011830201809
Test accuracy: 94.44%


### Part 3

**MLPClassifier**

In this part, we will use the same dataset from Part 2 and implement a multi-layer perceptron using sklearn. 
- Import the necessary classes and perform a 5-fold cross-validation by defining a hyperparameter grid for the MLP classifier. 
- You need to read about the hyperparameters supported by the function and define a grid for them.
- Perform a random search on the grid that you have chosen.
- Report a single test accuracy with the best found hyperparameters

**Note: you can use any sklearn function for this and the next part**

In [26]:
### Write your code here
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV

sklearn_mlp = MLPClassifier(max_iter=500)

In [27]:
#Creating Grid
grid = {
    'hidden_layer_sizes': [(50,30,20),(50,25,10),(25,10)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd'],
    'alpha': [0.001, 0.001, 0.01, 0.1],
}

#fit the grid
clf = RandomizedSearchCV(sklearn_mlp, grid, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=MLPClassifier(max_iter=500), n_jobs=-1,
                   param_distributions={'activation': ['tanh', 'relu'],
                                        'alpha': [0.001, 0.001, 0.01, 0.1],
                                        'hidden_layer_sizes': [(50, 30, 20),
                                                               (50, 25, 10),
                                                               (25, 10)],
                                        'solver': ['sgd']})

In [28]:
clf.best_params_

{'solver': 'sgd',
 'hidden_layer_sizes': (50, 30, 20),
 'alpha': 0.001,
 'activation': 'relu'}

**Result:**
The Neural Network with 3 layers with 50,30, and 20 neurons in each with activation function relu, trained with SGD with Learning Rate 0.001, gave the best accuracy

In [29]:
print('The Accuracy on Test Data is',accuracy_score(y_test,clf.predict(X_test)))

The Accuracy on Test Data is 0.9583333333333334


**MLPRegressor**

In this part, we would repeat the all steps taken for MLPClassifier. However, we will try to learn a regression model using MLPRegressor instead. In the end calculate the accuracy of MLPRegressor by using the *test_accuracy_regressor* function provided.

**Note: The target output needs to be numerical in this case.**

In [30]:
### Write your code here
from sklearn.neural_network import MLPRegressor
sklearn_mlp = MLPRegressor(max_iter=500)

#Creating Grid
grid = {
    'hidden_layer_sizes': [(50,30,20),(50,25,10),(25,10)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd'],
    'alpha': [0.001, 0.001, 0.01, 0.1],
}

#fit the grid
reg = RandomizedSearchCV(sklearn_mlp, grid, n_jobs=-1, cv=5)
reg.fit(X_train, np.argmax(y_train, axis=1))

RandomizedSearchCV(cv=5, estimator=MLPRegressor(max_iter=500), n_jobs=-1,
                   param_distributions={'activation': ['tanh', 'relu'],
                                        'alpha': [0.001, 0.001, 0.01, 0.1],
                                        'hidden_layer_sizes': [(50, 30, 20),
                                                               (50, 25, 10),
                                                               (25, 10)],
                                        'solver': ['sgd']})

In [31]:
reg.best_params_

{'solver': 'sgd',
 'hidden_layer_sizes': (50, 30, 20),
 'alpha': 0.01,
 'activation': 'tanh'}

**Result:**
The Neural Network with 3 layers with 50, 30, and 20 neurons in each with activation function tanh, trained with SGD with Learning Rate 0.01, gave the best accuracy

In [32]:
def test_accuracy_regressor(y_true,y_pred):
    ### the function assumes both inputs to be 1-D arrays
    assert y_true.shape==y_pred.shape, f"y_true and y_pred needs to be of same shape, but found y_true: {y_true.shape} and y_pred:{y_pred.shape}"
    assert len(y_pred.shape)==1, f'inputs should be 1-D, but found them as {len(y_pred.shape)}-D'

    return np.sum((np.round(y_pred,0).astype(int))==y_true)/y_pred.size

test_acc = test_accuracy_regressor(np.argmax(y_test, axis=1),reg.predict(X_test))
print('The Accuracy on Test Data is',test_acc)

The Accuracy on Test Data is 0.6333333333333333


Comment on the performance of MLP Regressor vs MLP Classifier

Classifier model gave the better accuracy because regression model treated the categorical target and numerical target.